In [1]:
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.path import Path
import numpy as np
import gsw
import warnings
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean
from collections import OrderedDict
import seaborn as sns
warnings.filterwarnings(action='once')
import os
import cartopy.feature as cfeature
import glob
from tqdm import tqdm as bar
from dask import delayed
import dask
from shapely.geometry import Polygon, Point, MultiPoint
from numba import jit
from dask.diagnostics import ProgressBar

In [2]:
# look first at the gps one (its a smaller dataset)
path = '/Users/manishdevana/Research/data_storage/surface_drifter_data/*.nc'
files = glob.glob(path)
files

gps = xr.open_dataset(files[0])

# if you want to include/exclude drogued drifters

/opt/anaconda3/envs/oceans/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:

# choose points in the zone you want
lons = gps.LON.values
lats = gps.LAT.values
times = gps.TIME.values
ids = gps.ID.values

# define a box
spg_lon = [5, 360-40]
spg_lat = [40, 85]

# mask based on box
lonmask = np.logical_and(lons>=spg_lon[0], lons<= spg_lon[1])
latmask = np.logical_and(lats>=spg_lat[0], lats<= spg_lat[1])

# masking
mask = np.logical_and(lonmask, latmask)
ids2 = ids[mask]

# The individual drifter float ids (indexing this way seems to be much quicker(not sure why  though))
drifters = np.unique(ids2) 
drifters = drifters[np.isfinite(drifters)]

# The new dataset
gps2 = gps.sel(TIME=gps.ID.isin(drifters))

# reorganize data into a list of datarrays where each array is a drifter tied to a single id
modified_drifts = []
for drifter in drifters:
    traj = gps2.sel(TIME=gps2.ID==drifter)
    dt = traj.TIME.diff(dim='TIME').values.astype(float)/1e9/60/60/24
#     step = np.ceil((len(dt)+1)/step_size).astype(int)
    
    trev = np.hstack((0,np.cumsum(dt)))
    new_ds = xr.Dataset({
        'lon':(['time'], traj.LON.values),
        'lat':(['time'], traj.LAT.values),
        # add u and v later
    },
        coords={
            'time':trev
        }
        
    )
    modified_drifts.append(new_ds)

/opt/anaconda3/envs/oceans/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# functions:

def prep_fields_and_trajs(modified_drift_set, data_dt = 1/24, resolution=0.25, lonlims=[], latlims=[]):
    """
    
    
    """
    
    # the drifter dataset is on a 0-360 longitude grid
    lon = np.arange(360+lonlims[0], 360+(lonlims[1]+resolution), resolution)
    lat = np.arange(latlims[0], latlims[1]+resolution, resolution)
    
    # Make a list of polygons using the lat long grid

    B = [] # This is the set of boxes B
    for k in range(1, lat.shape[0]):
        for i in range(1, lon.shape[0]):
             B.append([
                lon[i-1], lat[k-1],
                lon[i], lat[k]

            ])
                
    # put all the trajectories on to a uniform time grid starting at 0
    
    # make the tiem grid as long as the longest trajectory
    lens = []
    for d in modified_drift_set:
        lens.append(len(d.lon))

    max_l = np.nanmax(lens)
    trev = np.linspace(0, max_l*data_dt, max_l)
    
    # now nan pad all the trajecotories with nan to be same lenght
    drift_lon = []
    drift_lat = []
    for d in modified_drifts:
        nan_block = np.full(max_l - d.time.shape[0], np.nan)
        # adds nan to the end of the array 
        drift_lon.append(np.hstack((d.lon.values, nan_block)))
        drift_lat.append(np.hstack((d.lat.values, nan_block)))


    xx = np.array(drift_lon) # lon of drifters
    yy = np.array(drift_lat) # lat of drifters
    
    return B, xx, yy


def split_traj( xx, yy, T=1, data_dt=1/24):
    """
    # This splits all the trajectories into pairs seperated by T = timestep
    
    """
    
    subtraj = []
    steps = int(T/data_dt)
    for i in range(len(xx[:,0]) - steps):
        x0 = xx[:,i]
        y0 = yy[:,i]
        x1 = xx[:,i+steps]
        y1 = yy[:,i+steps]
        subtraj.append(np.stack([x0, y0, x1, y1]).T)


    subtraj = np.vstack(subtraj) # all subtrajectory pairs
    # clean up ones that have nans
    mask = np.sum(np.isfinite(subtraj), axis=1) == 4
    subtraj = subtraj[mask, :]
    
    return subtraj

    
    
# now compute the Probality Matrix values
def FindInside(box, points): 
    x = points[:, 0]
    y = points[:, 1]
    loncheck = np.logical_and(box[0] < x, box[2] > x)
    latcheck = np.logical_and(box[1] < y, box[3] > y)
    inside = np.logical_and(loncheck, latcheck)
    
    return inside

def _insideTest(box, x, y):
    
    loncheck = np.logical_and(box[0] < x, box[2] > x)
    
    latcheck = np.logical_and(box[1] < y, box[3] > y)
    inside = np.logical_and(loncheck, latcheck)
    if np.sum(inside) == 0:
        return False
    elif np.sum(inside) == 1:
        return True
    else:
        print('fuck up')
    
    


def TestInside(boxes, point):
    
    x = point[0]
    y = point[1]
    
    test = [_insideTest(box, x, y) for box in boxes]
    
    return int(np.where(test)[0])

    

def FindInsideCount(box, points): 
    x = points[:, 0]
    y = points[:, 1]
    loncheck = np.logical_and(box[0] < x, box[2] > x)
    latcheck = np.logical_and(box[1] < y, box[3] > y)
    inside = np.logical_and(loncheck, latcheck)
    
    return np.nansum(inside)



    
# @jit(parallel=True)
def FindNextCount(boxes, points):
    """
    
    
    """
    
    inside = np.array([FindInside(box,  points) for box in boxes]).T
    mask = np.sum(inside, axis=1)
    mask = mask == 1
    
    counts = np.sum(inside[mask, :], axis=0)
    
    #filter the ones that leave the domain
    
    # Ni revised to remove drifters that leave domain
    Ni = np.sum(mask)
    
    return counts, Ni
    
    

def construct_transit_matrix(B, subtraj):
    """
    
    
    """
    P1 = np.zeros((len(B), len(B))) # square matrix with dimension M = len(B)
    for i in bar(range(len(B))):
        Bij = B[i]
    #     points = 

        # first find all the points inside the current box
        inside = FindInside(Bij, subtraj[:, :2])

        # find the new box after one timestep from box i
        if np.sum(inside) > 0:

            counts, Ni = FindNextCount(B, subtraj[inside, 2:])

            # convert to probability
            if Ni > 0:
                Pij = counts/Ni
                if np.round((np.sum(Pij)), 1) != 1:
                    print('You messed up')
                    break
                P1[i,:] = Pij
                
        
    return P1
    
    

/opt/anaconda3/envs/oceans/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# do the transit matrix calculation

# Parameters:
resolution = 0.25
lonlims = [-40, 0]
latlims = [35, 85]
data_dt = 1/24 # i do the timesteps and time grids in decimal dayys and the drifter data is at 1 hour res
T = 2 # fixed timestep length for subtrajectoires
# do the prep
B, xx, yy = prep_fields_and_trajs(modified_drifts, 
                          data_dt=data_dt, 
                          resolution=resolution,
                          lonlims=lonlims,
                          
                                  latlims=latlims
                         )


# split into subtrajectories
subtrajectories = split_traj(xx, yy, T=T, data_dt=data_dt)


# final transit matrix
P = construct_transit_matrix(B, subtrajectories)


100%|██████████| 32000/32000 [03:53<00:00, 136.98it/s]


In [23]:
np.save('P_tester_0.25_res.npy', P)
np.save('boxes_0.25_res.npy',B,  allow_pickle=True)